In [1]:
import numpy as np
import random
from deap import base, creator, tools

In [2]:
def evaluate(individual):
    
    node = int(node_num[individual[0]])
    stime = individual[1]
    duration = individual[2]
    concentration = individual[3]
    fv = Fevaluate(node, stime, duration, concentration)

    return fv

def reevaluate(pop):
    d_share = 100
    fits = [ind.fitness.values[0] for ind in pop]
    mu = len(fits)
    dist = np.tile(0, (mu,mu))
    
    for i in range(mu):
        for j in range(mu):
            if pop[i][0] == pop[j][0]:
                d = (pop[i][1]-pop[j][1])**2 + (pop[i][2]-pop[j][2])**2 + (pop[i][3]-pop[j][3])**2
                if d < d_share:
                    dist[i][j] = 1-d/d_share
                    
    for i in range(mu):
        d = 0
        for j in range(mu):
            d += dist[i][j]
                
        fits[i] = fits[i] * d
    return fits
    

def select_elite(pop):
    INFMAX = 10000000
    fits = [ind.fitness.values[0] for ind in pop]
    index1 = np.argmin(fits)
    fits[index1] = INFMAX
    index2 = np.argmin(fits)
    
    return [pop[index1], pop[index2]]

In [3]:
def GA(mu = 40, generation = 100, niching = False):
    
    CXPB, MUTPB = 0.8, 0.8
    
    # Initialization
    pop = toolbox.population(mu)
    fitnesses = list(map(toolbox.evaluate, pop))
   
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = (fit,)

    fv = np.zeros(generation)
    g = 0
    while g < generation:
        
        # Generate a new generation
        offspring = toolbox.select(pop, k=len(pop))
        
        # Put two elites at the beginning
        offspring[0:2] = select_elite(pop)
        
        # Clone selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[2::2], offspring[3::2]):
            if random.random() < CXPB:
                toolbox.crossover(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring[2::1]:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluate individuals
        fitnesses = map(toolbox.evaluate, offspring)
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = (fit,)
       
        # Replace
        pop[:] = offspring
        
        fits = [ind.fitness.values[0] for ind in pop]    
        fv[g] = min(fits)
        
        # re-evaluate individuals by niching methods
        if niching == True:
            newfitnesses = reevaluate(pop)
            for ind, fit in zip(pop, newfitnesses):
                ind.fitness.values = (fit,)
        g = g + 1
    return pop[np.argmin(fits)][0], fv

In [4]:
from ctypes import *

libpath = "/home/lab1003/rk/epanet/source/linux/libs/network1/libepanet.so"
lib = cdll.LoadLibrary(libpath)
Finit = lib.ENFitnessInit
Fevaluate = lib.ENFitnessEvaluate
Fterminate = lib.ENFitnessTerminate
Fevaluate.restype = c_float

# Network 1

In [5]:
import scipy.io as sio

inpfile = "./ori.inp"
network1 = sio.loadmat("prior.mat")
predictions = network1['NODES']
nodes = network1['NODEID'][0]
reports = network1['REPORTS']

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("node", random.randint, 0, 91)
toolbox.register("stime", random.randint, 0, 23)
toolbox.register("duration", random.randint, 1, 23)
toolbox.register("concentration", random.randint, 5, 300)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.node, toolbox.stime, toolbox.duration, toolbox.concentration), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
toolbox.register("evaluate", evaluate)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=[0,0,1,5], up=[91, 23, 23, 300], indpb =0.5)
toolbox.register("select", tools.selTournament, tournsize=3)

## GA

In [7]:
node_num = nodes

n_events = 300

fv_ga = np.tile(0,(n_events, 100))
pred_ga = np.zeros(n_events)

tp = 0
for i in range(n_events): 
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_ga[i,:] = GA(40,100)
    Fterminate()
    pred_ga[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_ga', fv_ga)
np.save('pred_ga', pred_ga)

================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================
Node: 601 , Prediction: 123
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 159
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 60
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 109
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 120
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 119
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 123
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 139
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 257
===

Node: 105 , Prediction: 105
================  74 : ./report/221_10212_event_60_11_2_40.rpt  ===================
Node: 60 , Prediction: 127
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 163
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================
Node: 103 , Prediction: 193
================  77 : ./report/232_10222_event_263_10_1_280.rpt  ===================
Node: 263 , Prediction: 120
================  78 : ./report/238_10229_event_115_7_6_240.rpt  ===================
Node: 115 , Prediction: 115
================  79 : ./report/248_10239_event_101_7_14_80.rpt  ===================
Node: 101 , Prediction: 101
================  80 : ./report/249_1023_event_149_3_12_160.rpt  ===================
Node: 149 , Prediction: 151
================  81 : ./report/257_10247_event_119_21_1_220.rpt  ===================
Node: 119 , Prediction: 119
================  82 : ./report/259_10249_event_121_12_3_260.rpt  ==

Node: 10 , Prediction: 10
================  147 : ./report/453_10430_event_127_2_17_160.rpt  ===================
Node: 127 , Prediction: 151
================  148 : ./report/459_10436_event_149_10_2_210.rpt  ===================
Node: 149 , Prediction: 151
================  149 : ./report/460_10437_event_601_13_6_170.rpt  ===================
Node: 601 , Prediction: 601
================  150 : ./report/463_1043_event_61_13_10_260.rpt  ===================
Node: 61 , Prediction: 61
================  151 : ./report/470_10446_event_121_11_11_270.rpt  ===================
Node: 121 , Prediction: 121
================  152 : ./report/473_10449_event_157_15_2_270.rpt  ===================
Node: 157 , Prediction: 121
================  153 : ./report/478_10453_event_163_22_1_300.rpt  ===================
Node: 163 , Prediction: 159
================  154 : ./report/484_10459_event_263_9_15_150.rpt  ===================
Node: 263 , Prediction: 261
================  155 : ./report/486_10460_event_120_4_9

Node: 117 , Prediction: 117
================  219 : ./report/706_10673_event_125_15_9_40.rpt  ===================
Node: 125 , Prediction: 125
================  220 : ./report/707_10674_event_119_23_1_140.rpt  ===================
Node: 119 , Prediction: 60
================  221 : ./report/708_10676_event_129_1_23_90.rpt  ===================
Node: 129 , Prediction: 141
================  222 : ./report/709_10677_event_143_17_7_60.rpt  ===================
Node: 143 , Prediction: 60
================  223 : ./report/711_10679_event_139_6_10_300.rpt  ===================
Node: 139 , Prediction: 123
================  224 : ./report/712_1067_event_125_5_18_10.rpt  ===================
Node: 125 , Prediction: 109
================  225 : ./report/713_10680_event_61_7_6_210.rpt  ===================
Node: 61 , Prediction: 61
================  226 : ./report/716_10683_event_601_14_3_130.rpt  ===================
Node: 601 , Prediction: 601
================  227 : ./report/726_10692_event_119_18_3_200.r

Node: 61 , Prediction: 119
================  291 : ./report/913_10871_event_119_9_3_50.rpt  ===================
Node: 119 , Prediction: 119
================  292 : ./report/923_10882_event_257_9_10_230.rpt  ===================
Node: 257 , Prediction: 10
================  293 : ./report/925_10884_event_263_6_14_120.rpt  ===================
Node: 263 , Prediction: 119
================  294 : ./report/936_10894_event_193_7_7_270.rpt  ===================
Node: 193 , Prediction: 193
================  295 : ./report/939_10897_event_261_13_8_160.rpt  ===================
Node: 261 , Prediction: 117
================  296 : ./report/940_10898_event_123_16_5_220.rpt  ===================
Node: 123 , Prediction: 121
================  297 : ./report/945_10902_event_60_13_5_80.rpt  ===================
Node: 60 , Prediction: 60
================  298 : ./report/953_1090_event_120_3_1_120.rpt  ===================
Node: 120 , Prediction: 257
================  299 : ./report/957_10913_event_139_20_2_170.r

## niching GA

In [8]:
node_num = nodes

n_events = 300

fv_niching = np.tile(0,(n_events, 100))
pred_niching = np.zeros(n_events)

tp = 0
for i in range(n_events): 
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_niching[i,:] = GA(40,100,niching=True)
    Fterminate()
    pred_niching[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_niching', fv_niching)
np.save('pred_niching', pred_niching)

================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================
Node: 601 , Prediction: 60
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 123
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 60
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 109
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 120
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 60
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 123
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 139
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 259
=====

Node: 105 , Prediction: 257
================  74 : ./report/221_10212_event_60_11_2_40.rpt  ===================
Node: 60 , Prediction: 127
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 60
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================
Node: 103 , Prediction: 103
================  77 : ./report/232_10222_event_263_10_1_280.rpt  ===================
Node: 263 , Prediction: 263
================  78 : ./report/238_10229_event_115_7_6_240.rpt  ===================
Node: 115 , Prediction: 109
================  79 : ./report/248_10239_event_101_7_14_80.rpt  ===================
Node: 101 , Prediction: 259
================  80 : ./report/249_1023_event_149_3_12_160.rpt  ===================
Node: 149 , Prediction: 60
================  81 : ./report/257_10247_event_119_21_1_220.rpt  ===================
Node: 119 , Prediction: 60
================  82 : ./report/259_10249_event_121_12_3_260.rpt  =====

Node: 10 , Prediction: 10
================  147 : ./report/453_10430_event_127_2_17_160.rpt  ===================
Node: 127 , Prediction: 147
================  148 : ./report/459_10436_event_149_10_2_210.rpt  ===================
Node: 149 , Prediction: 61
================  149 : ./report/460_10437_event_601_13_6_170.rpt  ===================
Node: 601 , Prediction: 601
================  150 : ./report/463_1043_event_61_13_10_260.rpt  ===================
Node: 61 , Prediction: 61
================  151 : ./report/470_10446_event_121_11_11_270.rpt  ===================
Node: 121 , Prediction: 121
================  152 : ./report/473_10449_event_157_15_2_270.rpt  ===================
Node: 157 , Prediction: 157
================  153 : ./report/478_10453_event_163_22_1_300.rpt  ===================
Node: 163 , Prediction: 119
================  154 : ./report/484_10459_event_263_9_15_150.rpt  ===================
Node: 263 , Prediction: 125
================  155 : ./report/486_10460_event_120_4_9_

Node: 117 , Prediction: 117
================  219 : ./report/706_10673_event_125_15_9_40.rpt  ===================
Node: 125 , Prediction: 125
================  220 : ./report/707_10674_event_119_23_1_140.rpt  ===================
Node: 119 , Prediction: 119
================  221 : ./report/708_10676_event_129_1_23_90.rpt  ===================
Node: 129 , Prediction: 129
================  222 : ./report/709_10677_event_143_17_7_60.rpt  ===================
Node: 143 , Prediction: 139
================  223 : ./report/711_10679_event_139_6_10_300.rpt  ===================
Node: 139 , Prediction: 123
================  224 : ./report/712_1067_event_125_5_18_10.rpt  ===================
Node: 125 , Prediction: 113
================  225 : ./report/713_10680_event_61_7_6_210.rpt  ===================
Node: 61 , Prediction: 61
================  226 : ./report/716_10683_event_601_14_3_130.rpt  ===================
Node: 601 , Prediction: 601
================  227 : ./report/726_10692_event_119_18_3_200

Node: 61 , Prediction: 119
================  291 : ./report/913_10871_event_119_9_3_50.rpt  ===================
Node: 119 , Prediction: 119
================  292 : ./report/923_10882_event_257_9_10_230.rpt  ===================
Node: 257 , Prediction: 257
================  293 : ./report/925_10884_event_263_6_14_120.rpt  ===================
Node: 263 , Prediction: 263
================  294 : ./report/936_10894_event_193_7_7_270.rpt  ===================
Node: 193 , Prediction: 193
================  295 : ./report/939_10897_event_261_13_8_160.rpt  ===================
Node: 261 , Prediction: 117
================  296 : ./report/940_10898_event_123_16_5_220.rpt  ===================
Node: 123 , Prediction: 61
================  297 : ./report/945_10902_event_60_13_5_80.rpt  ===================
Node: 60 , Prediction: 121
================  298 : ./report/953_1090_event_120_3_1_120.rpt  ===================
Node: 120 , Prediction: 120
================  299 : ./report/957_10913_event_139_20_2_170.

## Our GA 

In [9]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("node", random.randint, 0, 4)
toolbox.register("stime", random.randint, 0, 23)
toolbox.register("duration", random.randint, 1, 23)
toolbox.register("concentration", random.randint, 5, 300)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.node, toolbox.stime, toolbox.duration, toolbox.concentration), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
toolbox.register("evaluate", evaluate)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=[0,0,1,5], up=[4, 23, 23, 300], indpb =0.5)
toolbox.register("select", tools.selRoulette)

n_events = 300
fv_our1 = np.tile(0,(n_events, 50))
pred_our1 = np.zeros(n_events)
tp = 0
for i in range(n_events): 
    node_num = predictions[i]
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_our1[i,:] = GA(40, 50)
    Fterminate()
    pred_our1[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_our1', fv_our1)
np.save('pred_our1', pred_our1)

================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================


/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Node: 601 , Prediction: 60
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 159
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 60
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 193
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 145
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 119
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 157
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 129
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 125
================  9 : ./report/27_10026_event_257_10_10_220.rpt  ===================
No

Node: 105 , Prediction: 257
================  74 : ./report/221_10212_event_60_11_2_40.rpt  ===================
Node: 60 , Prediction: 60
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 161
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================
Node: 103 , Prediction: 103
================  77 : ./report/232_10222_event_263_10_1_280.rpt  ===================
Node: 263 , Prediction: 263
================  78 : ./report/238_10229_event_115_7_6_240.rpt  ===================
Node: 115 , Prediction: 115
================  79 : ./report/248_10239_event_101_7_14_80.rpt  ===================
Node: 101 , Prediction: 101
================  80 : ./report/249_1023_event_149_3_12_160.rpt  ===================
Node: 149 , Prediction: 151
================  81 : ./report/257_10247_event_119_21_1_220.rpt  ===================
Node: 119 , Prediction: 60
================  82 : ./report/259_10249_event_121_12_3_260.rpt  ====

Node: 10 , Prediction: 10
================  147 : ./report/453_10430_event_127_2_17_160.rpt  ===================
Node: 127 , Prediction: 149
================  148 : ./report/459_10436_event_149_10_2_210.rpt  ===================
Node: 149 , Prediction: 149
================  149 : ./report/460_10437_event_601_13_6_170.rpt  ===================
Node: 601 , Prediction: 601
================  150 : ./report/463_1043_event_61_13_10_260.rpt  ===================
Node: 61 , Prediction: 61
================  151 : ./report/470_10446_event_121_11_11_270.rpt  ===================
Node: 121 , Prediction: 60
================  152 : ./report/473_10449_event_157_15_2_270.rpt  ===================
Node: 157 , Prediction: 157
================  153 : ./report/478_10453_event_163_22_1_300.rpt  ===================
Node: 163 , Prediction: 161
================  154 : ./report/484_10459_event_263_9_15_150.rpt  ===================
Node: 263 , Prediction: 263
================  155 : ./report/486_10460_event_120_4_9_

Node: 117 , Prediction: 117
================  219 : ./report/706_10673_event_125_15_9_40.rpt  ===================
Node: 125 , Prediction: 125
================  220 : ./report/707_10674_event_119_23_1_140.rpt  ===================
Node: 119 , Prediction: 60
================  221 : ./report/708_10676_event_129_1_23_90.rpt  ===================
Node: 129 , Prediction: 129
================  222 : ./report/709_10677_event_143_17_7_60.rpt  ===================
Node: 143 , Prediction: 147
================  223 : ./report/711_10679_event_139_6_10_300.rpt  ===================
Node: 139 , Prediction: 139
================  224 : ./report/712_1067_event_125_5_18_10.rpt  ===================
Node: 125 , Prediction: 259
================  225 : ./report/713_10680_event_61_7_6_210.rpt  ===================
Node: 61 , Prediction: 61
================  226 : ./report/716_10683_event_601_14_3_130.rpt  ===================
Node: 601 , Prediction: 601
================  227 : ./report/726_10692_event_119_18_3_200.

Node: 61 , Prediction: 61
================  291 : ./report/913_10871_event_119_9_3_50.rpt  ===================
Node: 119 , Prediction: 121
================  292 : ./report/923_10882_event_257_9_10_230.rpt  ===================
Node: 257 , Prediction: 257
================  293 : ./report/925_10884_event_263_6_14_120.rpt  ===================
Node: 263 , Prediction: 263
================  294 : ./report/936_10894_event_193_7_7_270.rpt  ===================
Node: 193 , Prediction: 193
================  295 : ./report/939_10897_event_261_13_8_160.rpt  ===================
Node: 261 , Prediction: 261
================  296 : ./report/940_10898_event_123_16_5_220.rpt  ===================
Node: 123 , Prediction: 123
================  297 : ./report/945_10902_event_60_13_5_80.rpt  ===================
Node: 60 , Prediction: 60
================  298 : ./report/953_1090_event_120_3_1_120.rpt  ===================
Node: 120 , Prediction: 120
================  299 : ./report/957_10913_event_139_20_2_170.r

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("node", random.randint, 0, 86)
toolbox.register("stime", random.randint, 0, 23)
toolbox.register("duration", random.randint, 1, 23)
toolbox.register("concentration", random.randint, 5, 300)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.node, toolbox.stime, toolbox.duration, toolbox.concentration), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
toolbox.register("evaluate", evaluate)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=[0,0,1,5], up=[4, 23, 23, 300], indpb =0.5)
toolbox.register("select", tools.selRoulette)

n_events = 300
fv_our2 = np.tile(0,(n_events, 50))
pred_our2 = np.zeros(n_events)
tp = 0
for i in range(n_events): 
    node_num = np.setdiff1d(nodes,predictions[i])
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_our2[i,:] = GA(40, 50)
    Fterminate()
    pred_our2[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_our2', fv_our2)
np.save('pred_our2', pred_our2)

================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================


/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Node: 601 , Prediction: 10
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 61
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 10
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 105
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 61
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 10
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 60
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 61
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 10
================  9 : ./report/27_10026_event_257_10_10_220.rpt  ===================
Node: 25

Node: 60 , Prediction: 20
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 61
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================


# Our Niching

In [7]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("node", random.randint, 0, 4)
toolbox.register("stime", random.randint, 0, 23)
toolbox.register("duration", random.randint, 1, 23)
toolbox.register("concentration", random.randint, 5, 300)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.node, toolbox.stime, toolbox.duration, toolbox.concentration), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
toolbox.register("evaluate", evaluate)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=[0,0,1,5], up=[4, 23, 23, 300], indpb =0.5)
toolbox.register("select", tools.selRoulette)

n_events = 300
fv_niching_our1 = np.tile(0,(n_events, 50))
pred_niching_our1 = np.zeros(n_events)
tp = 0
for i in range(n_events): 
    node_num = predictions[i]
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_niching_our1[i,:] = GA(40, 50,niching=True)
    Fterminate()
    pred_niching_our1[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_niching_our1', fv_niching_our1)
np.save('pred_niching_our1', pred_niching_our1)

/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================
Node: 601 , Prediction: 123
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 163
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 60
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 111
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 145
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 119
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 157
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 141
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 105
===

Node: 105 , Prediction: 105
================  74 : ./report/221_10212_event_60_11_2_40.rpt  ===================
Node: 60 , Prediction: 60
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 159
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================
Node: 103 , Prediction: 193
================  77 : ./report/232_10222_event_263_10_1_280.rpt  ===================
Node: 263 , Prediction: 263
================  78 : ./report/238_10229_event_115_7_6_240.rpt  ===================
Node: 115 , Prediction: 115
================  79 : ./report/248_10239_event_101_7_14_80.rpt  ===================
Node: 101 , Prediction: 101
================  80 : ./report/249_1023_event_149_3_12_160.rpt  ===================
Node: 149 , Prediction: 151
================  81 : ./report/257_10247_event_119_21_1_220.rpt  ===================
Node: 119 , Prediction: 119
================  82 : ./report/259_10249_event_121_12_3_260.rpt  ===

Node: 10 , Prediction: 10
================  147 : ./report/453_10430_event_127_2_17_160.rpt  ===================
Node: 127 , Prediction: 149
================  148 : ./report/459_10436_event_149_10_2_210.rpt  ===================
Node: 149 , Prediction: 149
================  149 : ./report/460_10437_event_601_13_6_170.rpt  ===================
Node: 601 , Prediction: 601
================  150 : ./report/463_1043_event_61_13_10_260.rpt  ===================
Node: 61 , Prediction: 61
================  151 : ./report/470_10446_event_121_11_11_270.rpt  ===================
Node: 121 , Prediction: 60
================  152 : ./report/473_10449_event_157_15_2_270.rpt  ===================
Node: 157 , Prediction: 157
================  153 : ./report/478_10453_event_163_22_1_300.rpt  ===================
Node: 163 , Prediction: 157
================  154 : ./report/484_10459_event_263_9_15_150.rpt  ===================
Node: 263 , Prediction: 261
================  155 : ./report/486_10460_event_120_4_9_

Node: 117 , Prediction: 117
================  219 : ./report/706_10673_event_125_15_9_40.rpt  ===================
Node: 125 , Prediction: 125
================  220 : ./report/707_10674_event_119_23_1_140.rpt  ===================
Node: 119 , Prediction: 121
================  221 : ./report/708_10676_event_129_1_23_90.rpt  ===================
Node: 129 , Prediction: 129
================  222 : ./report/709_10677_event_143_17_7_60.rpt  ===================
Node: 143 , Prediction: 143
================  223 : ./report/711_10679_event_139_6_10_300.rpt  ===================
Node: 139 , Prediction: 139
================  224 : ./report/712_1067_event_125_5_18_10.rpt  ===================
Node: 125 , Prediction: 259
================  225 : ./report/713_10680_event_61_7_6_210.rpt  ===================
Node: 61 , Prediction: 61
================  226 : ./report/716_10683_event_601_14_3_130.rpt  ===================
Node: 601 , Prediction: 601
================  227 : ./report/726_10692_event_119_18_3_200

Node: 61 , Prediction: 61
================  291 : ./report/913_10871_event_119_9_3_50.rpt  ===================
Node: 119 , Prediction: 119
================  292 : ./report/923_10882_event_257_9_10_230.rpt  ===================
Node: 257 , Prediction: 257
================  293 : ./report/925_10884_event_263_6_14_120.rpt  ===================
Node: 263 , Prediction: 263
================  294 : ./report/936_10894_event_193_7_7_270.rpt  ===================
Node: 193 , Prediction: 193
================  295 : ./report/939_10897_event_261_13_8_160.rpt  ===================
Node: 261 , Prediction: 261
================  296 : ./report/940_10898_event_123_16_5_220.rpt  ===================
Node: 123 , Prediction: 123
================  297 : ./report/945_10902_event_60_13_5_80.rpt  ===================
Node: 60 , Prediction: 60
================  298 : ./report/953_1090_event_120_3_1_120.rpt  ===================
Node: 120 , Prediction: 120
================  299 : ./report/957_10913_event_139_20_2_170.r

In [8]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("node", random.randint, 0, 86)
toolbox.register("stime", random.randint, 0, 23)
toolbox.register("duration", random.randint, 1, 23)
toolbox.register("concentration", random.randint, 5, 300)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.node, toolbox.stime, toolbox.duration, toolbox.concentration), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
toolbox.register("evaluate", evaluate)
toolbox.register("crossover", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=[0,0,1,5], up=[4, 23, 23, 300], indpb =0.5)
toolbox.register("select", tools.selRoulette)

n_events = 300
fv_niching_our2 = np.tile(0,(n_events, 50))
pred_niching_our2 = np.zeros(n_events)
tp = 0
for i in range(n_events): 
    node_num = np.setdiff1d(nodes,predictions[i])
    rptfile = "./report/"+reports[i].strip()
    truthnode = int(reports[i].strip().split('_')[3])
    print("================ ", i,":", rptfile, " ===================")
    Finit(bytes(inpfile,'utf-8'), bytes(rptfile, 'utf-8') )
    prednode, fv_niching_our2[i,:] = GA(40, 50,niching=True)
    Fterminate()
    pred_niching_our2[i] = node_num[prednode]
    if node_num[prednode] == truthnode:
        tp += 1
    print("Node:", truthnode,", Prediction:", node_num[prednode])
print("TP:", tp)
np.save('fv_niching_our2', fv_niching_our2)
np.save('pred_niching_our2', pred_niching_our2)

================  0 : ./report/1_10000_event_601_4_20_220.rpt  ===================


/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/lab1003/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Node: 601 , Prediction: 10
================  1 : ./report/3_10002_event_159_22_2_150.rpt  ===================
Node: 159 , Prediction: 61
================  2 : ./report/9_10008_event_60_15_7_210.rpt  ===================
Node: 60 , Prediction: 10
================  3 : ./report/12_10010_event_113_2_14_100.rpt  ===================
Node: 113 , Prediction: 10
================  4 : ./report/16_10014_event_145_5_10_40.rpt  ===================
Node: 145 , Prediction: 10
================  5 : ./report/19_10017_event_119_3_8_170.rpt  ===================
Node: 119 , Prediction: 60
================  6 : ./report/21_10020_event_157_13_5_80.rpt  ===================
Node: 157 , Prediction: 60
================  7 : ./report/22_10021_event_141_8_7_180.rpt  ===================
Node: 141 , Prediction: 20
================  8 : ./report/26_10025_event_257_11_3_30.rpt  ===================
Node: 257 , Prediction: 10
================  9 : ./report/27_10026_event_257_10_10_220.rpt  ===================
Node: 257

Node: 60 , Prediction: 20
================  75 : ./report/226_10217_event_161_13_7_70.rpt  ===================
Node: 161 , Prediction: 61
================  76 : ./report/230_10220_event_103_7_2_130.rpt  ===================
Node: 103 , Prediction: 10
================  77 : ./report/232_10222_event_263_10_1_280.rpt  ===================
Node: 263 , Prediction: 60
================  78 : ./report/238_10229_event_115_7_6_240.rpt  ===================
Node: 115 , Prediction: 10
================  79 : ./report/248_10239_event_101_7_14_80.rpt  ===================
Node: 101 , Prediction: 60
================  80 : ./report/249_1023_event_149_3_12_160.rpt  ===================
Node: 149 , Prediction: 60
================  81 : ./report/257_10247_event_119_21_1_220.rpt  ===================
Node: 119 , Prediction: 61
================  82 : ./report/259_10249_event_121_12_3_260.rpt  ===================
Node: 121 , Prediction: 10
================  83 : ./report/262_10251_event_120_2_15_170.rpt  =========

Node: 127 , Prediction: 60
================  148 : ./report/459_10436_event_149_10_2_210.rpt  ===================
Node: 149 , Prediction: 20
================  149 : ./report/460_10437_event_601_13_6_170.rpt  ===================
Node: 601 , Prediction: 15
================  150 : ./report/463_1043_event_61_13_10_260.rpt  ===================
Node: 61 , Prediction: 10
================  151 : ./report/470_10446_event_121_11_11_270.rpt  ===================
Node: 121 , Prediction: 61
================  152 : ./report/473_10449_event_157_15_2_270.rpt  ===================
Node: 157 , Prediction: 61
================  153 : ./report/478_10453_event_163_22_1_300.rpt  ===================
Node: 163 , Prediction: 60
================  154 : ./report/484_10459_event_263_9_15_150.rpt  ===================
Node: 263 , Prediction: 61
================  155 : ./report/486_10460_event_120_4_9_180.rpt  ===================
Node: 120 , Prediction: 60
================  156 : ./report/492_10467_event_157_21_1_180.r

Node: 119 , Prediction: 61
================  221 : ./report/708_10676_event_129_1_23_90.rpt  ===================
Node: 129 , Prediction: 40
================  222 : ./report/709_10677_event_143_17_7_60.rpt  ===================
Node: 143 , Prediction: 20
================  223 : ./report/711_10679_event_139_6_10_300.rpt  ===================
Node: 139 , Prediction: 10
================  224 : ./report/712_1067_event_125_5_18_10.rpt  ===================
Node: 125 , Prediction: 10
================  225 : ./report/713_10680_event_61_7_6_210.rpt  ===================
Node: 61 , Prediction: 10
================  226 : ./report/716_10683_event_601_14_3_130.rpt  ===================
Node: 601 , Prediction: 20
================  227 : ./report/726_10692_event_119_18_3_200.rpt  ===================
Node: 119 , Prediction: 20
================  228 : ./report/727_10693_event_109_5_17_240.rpt  ===================
Node: 109 , Prediction: 10
================  229 : ./report/729_10695_event_113_5_6_120.rpt  ==

Node: 263 , Prediction: 60
================  294 : ./report/936_10894_event_193_7_7_270.rpt  ===================
Node: 193 , Prediction: 10
================  295 : ./report/939_10897_event_261_13_8_160.rpt  ===================
Node: 261 , Prediction: 10
================  296 : ./report/940_10898_event_123_16_5_220.rpt  ===================
Node: 123 , Prediction: 20
================  297 : ./report/945_10902_event_60_13_5_80.rpt  ===================
Node: 60 , Prediction: 103
================  298 : ./report/953_1090_event_120_3_1_120.rpt  ===================
Node: 120 , Prediction: 105
================  299 : ./report/957_10913_event_139_20_2_170.rpt  ===================
Node: 139 , Prediction: 20
TP: 0
